<a href="https://colab.research.google.com/github/Denko-Sekka/Colab/blob/master/interactive_charts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## incase you need to isntall things
# !pip install -q keras

In [0]:
import pandas as pd
import numpy as np

## allows intellisense-like feel to the notebook
%config IPCompleter.greedy=True
##  whitespace cleaning
import re
# import webcolors ## package not in colab, need to check if supportable
from IPython.display import Image
##  histogram plotting
import matplotlib.pyplot as plt
##  3d plotting
from mpl_toolkits.mplot3d import Axes3D
## Used by matplotlib to remove duplicate legend handles
from collections import OrderedDict

## used for loading files
from google.colab import files

## used for loading files from google drive
from google.colab import drive


## used for interactive charts
import altair as alt

## converts the dictionary from google.colab's upload file into a stream
import io

# Choose File upload to Colab. 1. Upload file from computer. 2. Upload file from google drive.

In [19]:
# 1 Upload from file. Uncomment the code
# uploaded = files.upload() ## prompts csv upload
# print(type(uploaded[list(uploaded.keys())[0]]))

## the files are store as single entry dictionary
## to convert the file to a csv file and fed into read_csv, the values must be read.
## python dictionary keys/items does not allow indexing normally, you need import iteritems or convert it.
## Take the latter option so we convert it to a list and take the first and only entry.
## This value is class 'byte' it needs to be converted into an object that is readable by csv_read
## we convert it from bytes class to object using io.BytesIO and then fed to csv to be converted into a dataframe
# frame = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))

# ***********************************************************************************************************

# 2 Upload file from google drive. Uncomment the code
## follow the instructions and authenticate the google drive
drive.mount('/content/gdrive')

## From the left side tab of google colab, obtain the path to the csv
## Simply right click the csv file and copy path. Paste into csv_path
csv_path = '/content/gdrive/My Drive/BlueberryProject/csv files/softspotCrownMinus.csv'
frame = pd.read_csv(csv_path)
## if everything works, it should show the content
print(frame.head())
##  change the indexing to include column1(filenames)
# frame = frame.set_index([frame.index,frame.columns[0]])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
              FileName  Red_Mean  ...  SOFT2_Max  Blueberry Has Split Truth
0  190911 154418 5.gsi    22.929  ...     80.738                       True
1  190911 154422 2.gsi    21.121  ...     92.689                       True
2  190911 154426 5.gsi    17.068  ...     90.724                       True
3  190911 154429 7.gsi    23.214  ...     84.965                       True
4  190911 154435 7.gsi    18.426  ...     87.878                       True

[5 rows x 38 columns]


In [20]:
## verify that it works
frame.head(1)

,FileName,Red_Mean,Green_Mean,Blue_Mean,Hue_Mean,Sat_Mean,Lum_Mean,IR_Mean,SOFT_Mean,SOFT2_Mean,Red_Median,Green_Median,Blue_Median,Hue_Median,Sat_Median,Lum_Median,IR_Median,SOFT_Median,SOFT2_Median,Red_Min,Green_Min,Blue_Min,Hue_Min,Sat_Min,Lum_Min,IR_Min,SOFT_Min,SOFT2_Min,Red_Max,Green_Max,Blue_Max,Hue_Max,Sat_Max,Lum_Max,IR_Max,SOFT_Max,SOFT2_Max,Blueberry Has Split Truth
0,190911 154418 5.gsi,22.929,19.461,22.844,178.434,33.886,22.239,47.122,66.676,74.509,21.008,19.178,23.058,175.651,32.452,21.69,47.11,66.086,74.777,15.465,14.935,19.029,168.48,29.614,17.095,28.307,49.214,64.355,29.654,22.55,24.862,191.075,38.32,26.243,61.172,76.089,80.738,True


In [0]:
import math as math
## Add column names here to remove them
columnsToRemove = []

frame.head(2)
# append new features to the end of the frame
frame['Blueberry Has Split Truth'] = frame['Blueberry Has Split Truth']*1 # Convert T/F to 1/0
frame['SOFT2_MinMaxMean'] = (frame['SOFT2_Min'].add(frame['SOFT2_Max']))/2
frame['SOFT2_MinMaxRatio'] = frame['SOFT2_Min'].divide(frame['SOFT2_Max'])
frame['SOFT2_MeanXMin'] = frame['SOFT2_Mean'].add(frame['SOFT2_Min'])
frame['SatLogSOFT2'] = [math.log(frame['SOFT2_Max'][i],frame['Sat_Max'][i]) for i in range(len(frame['Blueberry Has Split Truth']))]
# frame['SOFT2_SatLogSOFT2'] = pd.DataFrame())

# Has Split Truth needs to be put back to the end of the columns, we pop it out and append it to the back end
frame['Blueberry Has Split Truth'] = frame.pop('Blueberry Has Split Truth')

## remove columns that were unused in the data
for c in columnsToRemove:
    
    if c in frame.columns:
        frame = frame.drop(c, 1)

In [22]:
select_n = frame.columns[1:len(frame.columns) - 1] ## select all the columns except if defect is true
print(select_n)
frame_n = frame[select_n] / frame[select_n].max(axis=0) ## normalize the values of every column
frame_n['Blueberry Has Split Truth'] = frame['Blueberry Has Split Truth']
print(frame_n)

Index(['Red_Mean', 'Green_Mean', 'Blue_Mean', 'Hue_Mean', 'Sat_Mean',
       'Lum_Mean', 'IR_Mean', 'SOFT_Mean', 'SOFT2_Mean', 'Red_Median',
       'Green_Median', 'Blue_Median', 'Hue_Median', 'Sat_Median', 'Lum_Median',
       'IR_Median', 'SOFT_Median', 'SOFT2_Median', 'Red_Min', 'Green_Min',
       'Blue_Min', 'Hue_Min', 'Sat_Min', 'Lum_Min', 'IR_Min', 'SOFT_Min',
       'SOFT2_Min', 'Red_Max', 'Green_Max', 'Blue_Max', 'Hue_Max', 'Sat_Max',
       'Lum_Max', 'IR_Max', 'SOFT_Max', 'SOFT2_Max', 'SOFT2_MinMaxMean',
       'SOFT2_MinMaxRatio', 'SOFT2_MeanXMin', 'SatLogSOFT2'],
      dtype='object')
     Red_Mean  Green_Mean  ...  SatLogSOFT2  Blueberry Has Split Truth
0    0.844935    0.913834  ...     0.862840                          1
1    0.778310    0.879414  ...     0.919928                          1
2    0.628957    0.760002  ...     0.951209                          1
3    0.855437    0.905475  ...     0.889087                          1
4    0.678999    0.736335  ...     0.826

# The data frame is split into their respective categories: Mean, Median, Min, Max. To be used for visualization purposes.

In [0]:
## Separate each of the features into its smaller data frames for better visualization
Mean = pd.DataFrame(frame[['Red_Mean','Blue_Mean', 'Green_Mean', 'Hue_Mean', 'Sat_Mean', 'Lum_Mean', 'IR_Mean', 'SOFT_Mean', 'SOFT2_Mean', 'SOFT2_MinMaxMean', 'SOFT2_MeanXMin','Blueberry Has Split Truth']], index = frame.index)
Median = pd.DataFrame(frame[['Red_Median', 'Blue_Median', 'Green_Median','Hue_Median','Sat_Median','Lum_Median','IR_Median','SOFT_Median','SOFT2_Median','Blueberry Has Split Truth']], index = frame.index)
Min = pd.DataFrame(frame[['Red_Min', 'Blue_Min', 'Green_Min', 'Hue_Min','Sat_Min','Lum_Min','IR_Min','SOFT_Min','SOFT2_Min', 'Blueberry Has Split Truth']], index = frame.index)
Max = pd.DataFrame(frame[['Red_Max', 'Blue_Max', 'Green_Max', 'Hue_Max','Sat_Max','Lum_Max','IR_Max','SOFT_Max','SOFT2_Max', 'Blueberry Has Split Truth']], index = frame.index)
Misc = pd.DataFrame(frame[['SatLogSOFT2',  'Blueberry Has Split Truth']], index = frame.index)

# Correlation statistics

In [0]:
# maskMean = np.triu(np.ones(Mean.corr().shape).astype(np.bool))
# corrMean = Mean.corr()

# maskMedian = np.triu(np.ones(Median.corr().shape).astype(np.bool))
# corrMedian = Median.corr()

# maskMin = np.triu(np.ones(Min.corr().shape).astype(np.bool))
# corrMin = Min.corr()

# maskMax = np.triu(np.ones(Max.corr().shape).astype(np.bool))
# corrMax = Max.corr()

# maskMisc = np.triu(np.ones(Misc.corr().shape).astype(np.bool))
# corrMisc = Misc.corr()

sorted_corr_mean = pd.DataFrame(Mean.corr()['Blueberry Has Split Truth'].iloc[Mean.corr()['Blueberry Has Split Truth'].abs().argsort()][::-1])
corr_mean = sorted_corr_mean[1:].transpose()

sorted_corr_median = pd.DataFrame(Median.corr()['Blueberry Has Split Truth'].iloc[Median.corr()['Blueberry Has Split Truth'].abs().argsort()][::-1])
corr_median = sorted_corr_median[1:].transpose()

sorted_corr_min = pd.DataFrame(Min.corr()['Blueberry Has Split Truth'].iloc[Min.corr()['Blueberry Has Split Truth'].abs().argsort()][::-1])
corr_min = sorted_corr_min[1:].transpose()

sorted_corr_max = pd.DataFrame(Max.corr()['Blueberry Has Split Truth'].iloc[Max.corr()['Blueberry Has Split Truth'].abs().argsort()][::-1])
corr_max = sorted_corr_max[1:].transpose()

sorted_corr_misc = pd.DataFrame(Misc.corr()['Blueberry Has Split Truth'].iloc[Misc.corr()['Blueberry Has Split Truth'].abs().argsort()][::-1])
corr_misc = sorted_corr_misc[1:].transpose()

# Ordered Correlation Tables

## Mean Correlation

In [25]:
# corrMean.mask(maskMean)
corr_mean

,SOFT2_Mean,SOFT2_MinMaxMean,SOFT2_MeanXMin,Sat_Mean,SOFT_Mean,Hue_Mean,Red_Mean,Blue_Mean,Green_Mean,IR_Mean,Lum_Mean
Blueberry Has Split Truth,-0.700781,-0.616826,-0.60425,0.537029,-0.374817,0.283613,0.212318,-0.127038,-0.06779,0.063961,0.013137


## Median Correlation

In [26]:
# corrMedian.mask(maskMedian)
corr_median

,SOFT2_Median,Sat_Median,SOFT_Median,Hue_Median,Red_Median,Blue_Median,Green_Median,IR_Median,Lum_Median
Blueberry Has Split Truth,-0.709589,0.437178,-0.344512,0.253254,0.173818,-0.122488,-0.072925,0.053848,0.002058


## Min Correlation

In [27]:
# corrMin.mask(maskMin)
corr_min

,SOFT2_Min,Sat_Min,SOFT_Min,Blue_Min,Green_Min,Lum_Min,Hue_Min,Red_Min,IR_Min
Blueberry Has Split Truth,-0.468008,0.342605,-0.20187,-0.153379,-0.116733,-0.073477,0.069381,0.040211,0.034879


## Max Correlation

In [28]:
# corrMax.mask(maskMax)
corr_max

,SOFT2_Max,Sat_Max,SOFT_Max,Hue_Max,Red_Max,Lum_Max,IR_Max,Blue_Max,Green_Max
Blueberry Has Split Truth,-0.685129,0.523454,-0.464953,0.414884,0.335006,0.125043,0.112862,-0.056275,0.025431


## Misc Correlation

In [29]:
# corrMisc.mask(maskMisc)
corr_misc

,SatLogSOFT2
Blueberry Has Split Truth,-0.674044


# Thresholding Chart

In [30]:
## set type interval
# choice = frame[frame_n['Lum_Mean'] <= frame_n['Blue_Mean']]
choice = frame_n

brush = alt.selection_interval()
n = len(choice[choice['Blueberry Has Split Truth'] == 1])
t = len(choice[choice['Blueberry Has Split Truth'] == 0])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    x = 'Red_Max',
    y = 'SOFT2_Max',
    color = alt.condition(brush, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush
)

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush
)

# Data Tables
test = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False_Count')
test2 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True_Count')
chart | (test & test2)


alt.HConcatChart(...)

### Manual labor

To calculate the confusion matrix, input the numbers below for <i>True</i> and <i>False</i>

In [31]:
TP, FP = 47, 1 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

recall good: 98.65%
recall bad: 74.6%
Precision Good: 97.92%
Precision Bad: 82.02%
Misclassification rate: 0.12%


#  <font color='gray'>Classification Plots</font>

## Mean Plot

In [32]:
## set type interval

choice = frame

brush = alt.selection_interval(encodings=['x'])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    y = 'Blueberry Has Split Truth:N',
    color = alt.condition(brush, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush
)

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush
)

print('Correlation')
rounding = 3
print([round(corrMean['Red_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['Blue_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['Green_Mean']['Blueberry Has Split Truth'], rounding)])
print([round(corrMean['Hue_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['Sat_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['Lum_Mean']['Blueberry Has Split Truth'], rounding)])
print([round(corrMean['IR_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['SOFT_Mean']['Blueberry Has Split Truth'], rounding),round(corrMean['SOFT2_Mean']['Blueberry Has Split Truth'], rounding)])
print([round(corrMean['SOFT2_MeanXMin']['Blueberry Has Split Truth'], rounding)])

m_0 = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False Count')
m_1 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True Count')

(chart.encode(x='Red_Mean:Q') | chart.encode(x='Blue_Mean:Q') | chart.encode(x='Green_Mean:Q')) & (chart.encode(x='Hue_Mean:Q') | chart.encode(x='Sat_Mean:Q') | chart.encode(x='Lum_Mean:Q'))  & (chart.encode(x='IR_Mean:Q') | chart.encode(x='SOFT_Mean:Q') | chart.encode(x='SOFT2_Mean:Q')) & chart.encode(x='SOFT2_MeanXMin:Q') | (m_0 & m_1)

Correlation


NameError: ignored

In [0]:
TP, FP = 24, 0 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

## Median Plot

In [0]:
## set type interval
choice = frame

brush2 = alt.selection_interval(encodings=['x'])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    y = 'Blueberry Has Split Truth:N',
    color = alt.condition(brush2, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush2
)

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush2
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush2
)

print('Correlation')
rounding = 3
print([round(corrMedian['Red_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['Blue_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['Green_Median']['Blueberry Has Split Truth'], rounding)])
print([round(corrMedian['Hue_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['Sat_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['Lum_Median']['Blueberry Has Split Truth'], rounding)])
print([round(corrMedian['IR_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['SOFT_Median']['Blueberry Has Split Truth'], rounding),round(corrMedian['SOFT2_Median']['Blueberry Has Split Truth'], rounding)])

m_0 = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False Count')
m_1 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True Count')

(chart.encode(x='Red_Median:Q') | chart.encode(x='Blue_Median:Q') | chart.encode(x='Green_Median:Q')) & (chart.encode(x='Hue_Median:Q') | chart.encode(x='Sat_Median:Q') | chart.encode(x='Lum_Median:Q'))  & (chart.encode(x='IR_Median:Q') | chart.encode(x='SOFT_Median:Q') | chart.encode(x='SOFT2_Median:Q')) | (m_0 & m_1)

In [0]:
TP, FP = 56, 21 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

# Min Plot

In [0]:
## set type interval
choice = frame
brush3 = alt.selection_interval(encodings=['x'])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    y = 'Blueberry Has Split Truth:N',
    color = alt.condition(brush3, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush3
)

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush3
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush3
)

print('Correlation')
rounding = 3
print([round(corrMin['Red_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['Blue_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['Green_Min']['Blueberry Has Split Truth'], rounding)])
print([round(corrMin['Hue_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['Sat_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['Lum_Min']['Blueberry Has Split Truth'], rounding)])
print([round(corrMin['IR_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['SOFT_Min']['Blueberry Has Split Truth'], rounding),round(corrMin['SOFT2_Min']['Blueberry Has Split Truth'], rounding)])

m_0 = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False Count')
m_1 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True Count')

(chart.encode(x='Red_Min:Q') | chart.encode(x='Blue_Min:Q') | chart.encode(x='Green_Min:Q')) & (chart.encode(x='Hue_Min:Q') | chart.encode(x='Sat_Min:Q') | chart.encode(x='Lum_Min:Q'))  & (chart.encode(x='IR_Min:Q') | chart.encode(x='SOFT_Min:Q') | chart.encode(x='SOFT2_Min:Q')) | (m_0 & m_1)

In [0]:
TP, FP = 39, 0 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

## Max Plot

In [0]:
## set type interval

choice = frame
brush4 = alt.selection_interval(encodings=['x'])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    y = 'Blueberry Has Split Truth:N',
    color = alt.condition(brush4, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush4
)

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush4
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush4
)

print('Correlation')
rounding = 3
print([round(corrMax['Red_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Blue_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Green_Max']['Blueberry Has Split Truth'], rounding)])
print([round(corrMax['Hue_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Sat_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Lum_Max']['Blueberry Has Split Truth'], rounding)])
print([round(corrMax['IR_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['SOFT_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['SOFT2_Max']['Blueberry Has Split Truth'], rounding)])

m_0 = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False Count')
m_1 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True Count')

(chart.encode(x='Red_Max:Q') | chart.encode(x='Blue_Max:Q') | chart.encode(x='Green_Max:Q')) & (chart.encode(x='Hue_Max:Q') | chart.encode(x='Sat_Max:Q') | chart.encode(x='Lum_Max:Q'))  & (chart.encode(x='IR_Max:Q') | chart.encode(x='SOFT_Max:Q') | chart.encode(x='SOFT2_Max:Q')) | (m_0 & m_1)

In [0]:
TP, FP = 56, 21 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

In [0]:
## set type interval

choice = frame
brush1 = alt.selection_interval(
    on="[mousedown[event.altKey], mouseup] > mousemove",
    name='brush1',
    encodings = ['x']
)
brush2 = alt.selection_interval(
    on="[mousedown[event.shiftKey], mouseup] > mousemove",
    mark=alt.BrushConfig(fill="#fdbb84", fillOpacity=0.5, stroke="#e34a33"),
    name='brush2',
    encodings = ['x']
)
brush3 = alt.selection_interval(
    on="[mousedown[event.ctrlKey], mouseup] > mousemove",
    encodings=['x'],
    mark=alt.BrushConfig(fill="#757575", fillOpacity=0.5, stroke="#e34a33")
  )
# brush4 = alt.selection_interval(encodings=['x'])
# brush5 = alt.selection_interval(encodings=['x'])

chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
    y = 'Blueberry Has Split Truth:N',
    color = alt.condition(brush1 & brush2 & brush3, 'Blueberry Has Split Truth:N', alt.value('lightgray'))
).add_selection(
    brush1, brush2, brush3
)

# chart = alt.Chart(choice).mark_circle(opacity = 0.2, size = 100).encode(
#     y = 'Blueberry Has Split Truth:N'
# )

tbrush1 = alt.selection_interval(encodings = ['x'])
tbrush2 = alt.selection_interval(encodings = ['x'])
tbrush3 = alt.selection_interval(encodings = ['x'])

# Base chart for Matrix
fp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 0]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush1 & brush2 & brush3
)

tp = alt.Chart(choice[choice['Blueberry Has Split Truth'] == 1]).mark_text().encode(
  y=alt.Y('count:O',axis=None)
).transform_filter(
    brush1 & brush2 & brush3
)

# print('Correlation')
# rounding = 3
# print([round(corrMax['Red_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Blue_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Green_Max']['Blueberry Has Split Truth'], rounding)])
# print([round(corrMax['Hue_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Sat_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['Lum_Max']['Blueberry Has Split Truth'], rounding)])
# print([round(corrMax['IR_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['SOFT_Max']['Blueberry Has Split Truth'], rounding),round(corrMax['SOFT2_Max']['Blueberry Has Split Truth'], rounding)])

m_0 = fp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='False Count')
m_1 = tp.encode(text = 'count():N', key = 'Blueberry Has Split Truth').properties(title='True Count')

cmax = (chart.encode(x='Red_Max:Q') | chart.encode(x='Blue_Max:Q') | chart.encode(x='Green_Max:Q')) & (chart.encode(x='Hue_Max:Q') | chart.encode(x='Sat_Max:Q') | chart.encode(x='Lum_Max:Q'))  & (chart.encode(x='IR_Max:Q') | chart.encode(x='SOFT_Max:Q') | chart.encode(x='SOFT2_Max:Q'))
cmin = (chart.encode(x='Red_Min:Q') | chart.encode(x='Blue_Min:Q') | chart.encode(x='Green_Min:Q')) & (chart.encode(x='Hue_Min:Q') | chart.encode(x='Sat_Min:Q') | chart.encode(x='Lum_Min:Q'))  & (chart.encode(x='IR_Min:Q') | chart.encode(x='SOFT_Min:Q') | chart.encode(x='SOFT2_Min:Q'))
cmedian = (chart.encode(x='Red_Median:Q') | chart.encode(x='Blue_Median:Q') | chart.encode(x='Green_Median:Q')) & (chart.encode(x='Hue_Median:Q') | chart.encode(x='Sat_Median:Q') | chart.encode(x='Lum_Median:Q'))  & (chart.encode(x='IR_Median:Q') | chart.encode(x='SOFT_Median:Q') | chart.encode(x='SOFT2_Median:Q'))
cmean = (chart.encode(x='Red_Mean:Q') | chart.encode(x='Blue_Mean:Q') | chart.encode(x='Green_Mean:Q')) & (chart.encode(x='Hue_Mean:Q') | chart.encode(x='Sat_Mean:Q') | chart.encode(x='Lum_Mean:Q'))  & (chart.encode(x='IR_Mean:Q') | chart.encode(x='SOFT_Mean:Q') | chart.encode(x='SOFT2_Mean:Q')) & chart.encode(x='SOFT2_MeanXMin:Q')
cmisc = chart.encode(x = 'SatLogSOFT2:Q')

(cmax & cmin & cmedian & cmean & cmisc & (m_0 & m_1)) | (m_0 & m_1)

# testmax = chart.encode(x='Red_Max:Q', color = alt.condition(tbrush1, 'Blueberry Has Split Truth:N', alt.value('lightgray'))).add_selection(tbrush1) | \
#   chart.encode(x='Blue_Max:Q', color = alt.condition(tbrush2, 'Blueberry Has Split Truth:N', alt.value('lightgray'))).add_selection(tbrush2)
# testmax & (m_0 | m_1)

In [0]:
TP, FP = 48, 0 # Replace these two numbers with the numbers calculated below
total_T, total_F = len(frame_n[frame_n['Blueberry Has Split Truth'] == 1]), len(frame_n[frame_n['Blueberry Has Split Truth'] == 0])

if(TP == 0 and FP == 0):
  print('TP and FP are set to 0, cannot continue')
else:
  rounding = 2
  FN, TN = total_T - TP, total_F - FP

  # Recall Good: How many of the good is classified as good?
  # TP/total_T
  r_bad = round(TP/total_T*100, rounding)

  # Recall Bad: How many of the bad is classified as bad?
  # TN/total_N
  r_good = round(TN/total_F*100, rounding)

  # Precision Good: When it predicts yes, how often is it correct? 
  # TP / (TP + FP)
  p_good = round(TP/(TP+FP)*100, rounding)

  # Precision Bad: When it predicts no, how often is it correct? 
  # TN / (TN + FN)
  p_bad = round(TN/(TN+FN)*100, rounding)

  # Misclassification Rate: Percentage of misclassified
  # (FN + FP)/ (total_T + total_F)
  m_rate = round((FN + FP)/(total_T + total_F), rounding)

  print('recall good: ' + str(r_good) + '%')
  print('recall bad: ' + str(r_bad)+ '%')
  print('Precision Good: ' + str(p_good)+ '%')
  print('Precision Bad: ' + str(p_bad)+ '%')
  print('Misclassification rate: ' + str(m_rate)+ '%')

# Random Forest Tree

In [0]:
# import Library
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
r_state = 10 ## given random_state to sklearn methods

In [0]:
##  Get Model Instantiation
##  n_estimators -> # of trees in the forest
##  random_state -> allows the values to be recreated every run
##  oob_score -> uses out-of-bag samples
RandFor = RandomForestClassifier(n_estimators = 100, oob_score = 1, random_state = r_state)

## Even Distribution

In [0]:
minCount = min(len(Mean[Mean['Blueberry Has Split Truth'] == 1]), len(Mean[Mean['Blueberry Has Split Truth'] == 0]))
X_1 = frame[frame['Blueberry Has Split Truth'] == 1][0:minCount]
X_0 = frame[frame['Blueberry Has Split Truth'] == 0][0:minCount]

In [0]:
X_Even = pd.concat([X_1,X_0])

In [0]:
target = 'Blueberry Has Split Truth'
if 'Y_Even' not in globals():
    Y_Even = X_Even.pop(target)

In [0]:
##  Pick a combination of features that maximize results
features = ['SOFT2_Min', 'Sat_Mean', 'Sat_Max', 'SOFT2_Mean']
X_train, X_test, Y_train, Y_test = train_test_split(X_Even[features],Y_Even, test_size = 0.3, random_state =  r_state)

In [0]:
##  Fit our testing data/result to the model
RandFor.fit(X_train,Y_train)

##  Get our predictions from the X_test
Y_pred = RandFor.predict(X_test)

In [0]:
from sklearn import metrics

In [0]:
cm = metrics.confusion_matrix(Y_test, Y_pred)
TP = cm[1,1]
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
cm

In [0]:
recall = metrics.recall_score(Y_test, Y_pred)
print('recall good: ' + str(recall)) ##  same as metrics.recall_score
sensitivity = TN / (TN + FP)
print('recall bad: ' + str(sensitivity))
precision1 = TP / (TP + FP) ##  same as metrics.precision_score, 0 positive rate
print('precision good: ' + str(precision1))
precision2 = TN / (TN + FN) ##  specificity
print('precision bad: ' + str(precision2))

In [0]:
print('accuracy score: ' + str(metrics.accuracy_score(Y_test, Y_pred))) ## how correct is the classifier
print('Class Distibution: ' + str(max(Y_test.mean(), 1-Y_test.mean()))) ## distribution of most frequent result divided # total result
print('Classification error rate: ' + str((FP + FN)/ (FP + FN + TP + TN))) ## How frequent does the misidentify
print('0 Positive Rate: ' + str(FP/(FP + TN)))
print('0 Negative Rate: ' + str(FN/(FN + TP)))

## Uneven Distribution

In [0]:
RandFor2 = RandomForestClassifier(n_estimators = 100, oob_score = 1, random_state = r_state)
target = 'Blueberry Has Split Truth'
X_copy = frame.copy()
Y = X_copy.pop(target)
X_train, X_test, Y_train, Y_test = train_test_split(X_copy[features],Y, test_size = 0.3, random_state =  r_state)

##  Fit our testing data/result to the model
RandFor2.fit(X_train,Y_train)

##  Get our predictions from the X_test
Y_pred = RandFor2.predict(X_test)

In [0]:
X_copy.describe()

In [0]:
cm = metrics.confusion_matrix(Y_test, Y_pred)
TP = cm[1,1]
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
cm

In [0]:
recall = metrics.recall_score(Y_test, Y_pred)
print('recall good: ' + str(recall)) ##  same as metrics.recall_score
sensitivity = TN / (TN + FP)
print('recall bad: ' + str(sensitivity))
precision1 = TP / (TP + FP) ##  same as metrics.precision_score, 0 positive rate
print('precision good: ' + str(precision1))
precision2 = TN / (TN + FN) ##  specificity
print('precision bad: ' + str(precision2))

In [0]:
print('accuracy score: ' + str(metrics.accuracy_score(Y_test, Y_pred))) ## how correct is the classifier
print('Class Distibution: ' + str(max(Y_test.mean(), 1-Y_test.mean()))) ## distribution of most frequent result divided # total result
print('Classification error rate: ' + str((FP + FN)/ (FP + FN + TP + TN))) ## How frequent does the misidentify
print('0 Positive Rate: ' + str(FP/(FP + TN)))
print('0 Negative Rate: ' + str(FN/(FN + TP)))

# Example Altair

In [0]:
import altair as alt
from vega_datasets import data

cars = data.cars.url

### Conditional variables with datum

In [0]:
import altair as alt
from vega_datasets import data

def plot_column(col_name: str) -> alt.Chart:
    source = data.us_employment()

    return alt.Chart(source).mark_bar().encode(
      x="month:T",
      y=f"{col_name}:Q", ## f is probably similar to C# version of $'{_variablename}'
      color=alt.condition(
          f"datum.{col_name} > 0", ## datum refers to the data source object given to chart
          alt.value("steelblue"),  # The positive color
          alt.value("orange")  # The negative color
      )
    ).properties(width=600)

plot_column("nonfarm_change")

### Custom Mouse Events

In [0]:
alex = alt.selection_interval(
    on="[mousedown[event.altKey], mouseup] > mousemove",
    name='alex'
)
morgan = alt.selection_interval(
    on="[mousedown[event.shiftKey], mouseup] > mousemove",
    mark=alt.BrushConfig(fill="#fdbb84", fillOpacity=0.5, stroke="#e34a33"),
    name='morgan'
)

alt.Chart(cars).mark_rect().encode(
    x='Cylinders:O',
    y='Origin:O',
    color=alt.condition(alex | morgan, 'count()', alt.ColorValue("grey"))
).add_selection(
    alex, morgan
).properties(
    width=300,
    height=180
)